## Assignment: Segmenting and Clustering Neighborhoods in Toronto

For this assignment, it is required to explore and cluster the neighborhoods in Toronto. So, let's start by creating a new Notebook for this assignment.

I will use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

### Step 1: Import Libraries ###

In [2]:
!pip install beautifulsoup4 -U
!pip install lxml -U
!pip install requests -U
!pip install html5lib -U
!pip install urllib3 -U
!pip install geocoder -U

Requirement already up-to-date: beautifulsoup4 in /opt/conda/envs/Python36/lib/python3.6/site-packages (4.8.1)
Requirement already up-to-date: lxml in /opt/conda/envs/Python36/lib/python3.6/site-packages (4.4.1)
Requirement already up-to-date: requests in /opt/conda/envs/Python36/lib/python3.6/site-packages (2.22.0)
Requirement already up-to-date: html5lib in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.0.1)
Requirement already up-to-date: urllib3 in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.25.7)
Requirement already up-to-date: geocoder in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.38.1)


In [3]:
import html.parser
from bs4 import BeautifulSoup
import requests
import urllib3.request
import pandas as pd
import numpy as np

### Step 2: Scraping the table from wikipedia ###

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(response, 'lxml')
table = soup.find('div', id = 'mw-content-text')
rows = table.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]
df = pd.DataFrame(columns=columns)
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n','')]
    else:
        pass
    df = df.append(pd.Series(values, index = columns), ignore_index=True)
    df.to_csv('toronto.csv')
df = pd.read_csv('toronto.csv')
df.shape

(292, 4)

In [5]:
df.head(10)

,Unnamed: 0,Postcode,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,Harbourfront
5,5,M6A,North York,Lawrence Heights
6,6,M6A,North York,Lawrence Manor
7,7,M7A,Queen's Park,Not assigned
8,8,M8A,Not assigned,Not assigned
9,9,M9A,Queen's Park,Queen's Park


### Step 3: Create DataFrame ###

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. Only to process the cells that have an assigned borough and ignore cells with a borough that is Not assigned below code has been used.

In [7]:
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
df['Borough'].replace('Not assigned',np.nan,inplace = True)
df.dropna(subset=['Borough'], axis = 0, inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()
df_grouped = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.tail(20)
for index, row in df_grouped.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
df_grouped

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
df_grouped.shape

(103, 3)